In [1]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def preprocess_data(X, Y):
    X = K.applications.densenet.preprocess_input(X)
    Y = K.utils.to_categorical(Y)
    return X, Y

In [3]:
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

170508288/170498071 [==============================] - 13s 0us/step


In [4]:
initializer = K.initializers.he_normal()
input_tensor = K.Input(shape=(32, 32, 3))

resized_images = K.layers.Lambda(lambda image: tf.image.resize(image, (224, 224)))(input_tensor)

model = K.applications.ResNet50(include_top=False,
                                   weights='imagenet',
                                   input_tensor=resized_images,
                                   input_shape=(224, 224, 3),
                                   pooling='max',
                                   classes=1000)

94781440/94765736 [==============================] - 1s 0us/step


In [5]:
for layer in model.layers:
    layer.trainable = False
output = model.layers[-1].output
flatten = K.layers.Flatten(name='feats')
output = flatten(output)
model = K.models.Model(inputs=input_tensor, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 lambda (Lambda)                (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['lambda[0][0]']                 
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                             

In [6]:
y_test.shape

(10000, 10)

In [7]:
preds=model.predict(x_train)
preds.shape

(50000, 2048)

In [8]:
preds_test=model.predict(x_test)
preds_test.shape

(10000, 2048)

In [9]:
import copy
real_preds=copy.deepcopy(preds)

In [10]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=42, max_iter=1000, algorithm='elkan').fit(real_preds)

preds=kmeans.predict(preds_test) # use features extracted from denseNet
preds.shape

(10000,)

In [11]:
y1=np.argmax(y_test,axis=1)
y1.shape

(10000,)

In [13]:
# assign predicted clusters to actual classes to get the accuracy
from collections import Counter

correct=0
mapping={}

for val in set(y1):
  inds=[i for i in range(len(y1)) if y1[i]==val] # all indices
  p=preds[inds]
  y2=y1[inds]
  counts=dict(Counter(p))

  print("y2: ", y2[:2])
  print("counts: ", counts)
  print("----------------")

y2:  [0 0]
counts:  {7: 182, 1: 66, 9: 157, 5: 250, 2: 113, 3: 32, 4: 43, 6: 86, 8: 39, 0: 32}
----------------
y2:  [1 1]
counts:  {4: 209, 7: 244, 5: 172, 6: 160, 1: 56, 9: 44, 2: 25, 8: 39, 0: 44, 3: 7}
----------------
y2:  [2 2]
counts:  {8: 120, 6: 127, 5: 81, 9: 96, 2: 202, 1: 200, 3: 93, 0: 41, 7: 29, 4: 11}
----------------
y2:  [3 3]
counts:  {6: 185, 1: 232, 8: 156, 7: 18, 0: 116, 9: 24, 2: 155, 5: 47, 3: 44, 4: 23}
----------------
y2:  [4 4]
counts:  {2: 247, 8: 147, 5: 60, 4: 14, 6: 159, 1: 252, 0: 59, 3: 36, 9: 10, 7: 16}
----------------
y2:  [5 5]
counts:  {8: 144, 6: 207, 1: 241, 2: 93, 7: 4, 0: 187, 5: 34, 3: 61, 9: 9, 4: 20}
----------------
y2:  [6 6]
counts:  {1: 270, 2: 139, 8: 85, 6: 276, 5: 61, 0: 51, 9: 61, 4: 35, 7: 18, 3: 4}
----------------
y2:  [7 7]
counts:  {3: 100, 6: 173, 4: 66, 1: 155, 0: 216, 5: 29, 9: 14, 8: 187, 2: 47, 7: 13}
----------------
y2:  [8 8]
counts:  {5: 341, 6: 78, 2: 184, 7: 198, 1: 57, 4: 38, 9: 62, 8: 19, 3: 10, 0: 13}
-------------

In [14]:
#resnet
mapping={
    0:5,
    1:7,
    2:2,
    3:6,
    4:8,
    5:1,
    6:6,
    7:3,
    8:5,
    9:4
}
correct=0
for val in set(y1):
  inds=[i for i in range(len(y1)) if y1[i]==val] # all indices
  p=preds[inds]
  y2=y1[inds]
  counts=dict(Counter(p))
  correct+=counts[mapping[y2[0]]]
print("accuracy: ", correct*100/len(y1))

accuracy:  22.47
